In [9]:
# 他人图片集下载：http://vis-www.cs.umass.edu/lfw/lfw.tgz

import tensorflow as tf
from img_to_tf_record import Img2TFRecord


my_faces_path = './my_faces'
other_faces_path = './other_faces'
size = 64

imgs = []
labs = []


x = tf.placeholder(tf.float32, [None, 250, 151, 1])  #[None, size, size, 3] Height, Width, channel
y_ = tf.placeholder(tf.float32, [None, 2])

keep_prob_5 = tf.placeholder(tf.float32)
keep_prob_75 = tf.placeholder(tf.float32)

def weightVariable(shape):
    init = tf.random_normal(shape, stddev = 0.01)
    return tf.Variable(init)

def biasVariable(shape):
    init = tf.random_normal(shape)
    return tf.Variable(init)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides = [1, 1, 1, 1], padding = 'SAME')

def maxPool(x):
    return tf.nn.max_pool(x, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')

def dropout(x, keep):
    return tf.nn.dropout(x, keep)

def cnnLayer():
    '''
    使用tf创建3层cnn，3 * 3的filter，输入为rgb所以：

    第一层的channel是3，图像宽高为64，输出32个filter，maxpooling是缩放一倍
    第二层的输入为32个channel，宽高是32，输出为64个filter，maxpooling是缩放一倍
    第三层的输入为64个channel，宽高是16，输出为64个filter，maxpooling是缩放一倍

    所以最后输入的图像是8 * 8 * 64，卷积层和全连接层都设置了dropout参数

    将输入的8 * 8 * 64的多维度，进行flatten，映射到512个数据上，然后进行softmax，输出到onehot类别上，类别的输入根据采集的人员的个数来确定。
    '''   
    
    # 第一层
    W1 = weightVariable([3, 3, 1, 32]) # 卷积核大小(3,3)， 输入通道(3)， 输出通道(32)
    b1 = biasVariable([32])
    # 卷积
    conv1 = tf.nn.relu(conv2d(x, W1) + b1)
    # 池化
    pool1 = maxPool(conv1)
    # 减少过拟合，随机让某些权重不更新
    drop1 = dropout(pool1, keep_prob_5)

    # 第二层
    W2 = weightVariable([3, 3, 32, 64])
    b2 = biasVariable([64])
    conv2 = tf.nn.relu(conv2d(drop1, W2) + b2)
    pool2 = maxPool(conv2)
    drop2 = dropout(pool2, keep_prob_5)

    # 第三层
    W3 = weightVariable([3, 3, 64, 64])
    b3 = biasVariable([64])
    conv3 = tf.nn.relu(conv2d(drop2, W3) + b3)
    pool3 = maxPool(conv3)
    drop3 = dropout(pool3, keep_prob_5)

    # 全连接层
    Wf = weightVariable([8 * 8 * 64, 512])
    bf = biasVariable([512])
    drop3_flat = tf.reshape(drop3, [-1, 8 * 8 * 64])
    dense = tf.nn.relu(tf.matmul(drop3_flat, Wf) + bf)
    dropf = dropout(dense, keep_prob_75)

    # 输出层
    Wout = weightVariable([512, 2])
    bout = weightVariable([2])
    #out = tf.matmul(dropf, Wout) + bout
    out = tf.add(tf.matmul(dropf, Wout), bout)
    return out

def cnnTrain():
    out = cnnLayer()
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = out, labels = y_))
    train_step = tf.train.AdamOptimizer(0.01).minimize(cross_entropy)
    # 比较标签是否相等，再求的所有数的平均值，tf.cast(强制转换类型)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(out, 1), tf.argmax(y_, 1)), tf.float32))
    # 将loss与accuracy保存以供tensorboard使用
    tf.summary.scalar('loss', cross_entropy)
    tf.summary.scalar('accuracy', accuracy)
    merged_summary_op = tf.summary.merge_all()
    # 数据保存器的初始化
    saver = tf.train.Saver()

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        summary_writer = tf.summary.FileWriter('./tmp', graph = tf.get_default_graph())
        for n in range(10):
             # 每次取128(batch_size)张图片
            for i in range(num_batch):
                batch_x = train_x[i*batch_size : (i + 1) * batch_size]
                batch_y = train_y[i*batch_size : (i + 1) * batch_size]
                # 开始训练数据，同时训练三个变量，返回三个数据
                _, loss, summary = sess.run([train_step, cross_entropy, merged_summary_op],
                                           feed_dict = {x:batch_x, y_:batch_y, keep_prob_5:0.5, keep_prob_75:0.75})
                summary_writer.add_summary(summary, n * num_batch + i)
                # 打印损失
                # print("loss ", n*num_batch + i, loss)

                if (n * num_batch + i) % 100 == 0:
                    # 获取测试数据的准确率
                    acc = accuracy.eval({x:test_x, y_:test_y, keep_prob_5:1.0, keep_prob_75:1.0})
                    print(n * num_batch + i, "acc:", acc, "  loss:", loss)
                    # 准确率大于0.98时保存并退出
                    if acc > 0.98 and n > 2:
                        saver.save(sess, model_path + '/train_faces.model', global_step = n * num_batch + i)
                        sys.exit(0)
        print('accuracy less 0.98, exited!')

def validate(test_x, tfsavepath): 
    ''' validate ''' 
    output = cnnLayer(2) 
    #predict = tf.equal(tf.argmax(output, 1), tf.argmax(y_data, 1)) 
    predict = output 
    saver = tf.train.Saver() 
    
    with tf.Session() as sess: 
        #sess.run(tf.global_variables_initializer()) 
        saver.restore(sess, tfsavepath) 
        res = sess.run([predict, tf.argmax(output, 1)], 
                       feed_dict={x_data: test_x, keep_prob_5:1.0, keep_prob_75: 1.0}) 
        return res




if __name__ == '__main__':
    one_Set = Img2TFRecord('./imagenet-dogs', '/home/yangyuqi/work/tf_algorithm_example/out/dogs_record')

    # test renerate function
    # one_Set.generate_tf_record_files( (250, 151) )
    
    
    out = cnnLayer()
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = out, labels = y_))
    train_step = tf.train.AdamOptimizer(0.01).minimize(cross_entropy)
    # 比较标签是否相等，再求的所有数的平均值，tf.cast(强制转换类型)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(out, 1), tf.argmax(y_, 1)), tf.float32))
    # 将loss与accuracy保存以供tensorboard使用
    tf.summary.scalar('loss', cross_entropy)
    tf.summary.scalar('accuracy', accuracy)
    merged_summary_op = tf.summary.merge_all()
    # 数据保存器的初始化
    saver = tf.train.Saver()

    
    image_batch, label_batch = one_Set.read_train_images_from_tf_records([250, 151, 1], 5)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        summary_writer = tf.summary.FileWriter('./tmp', graph = tf.get_default_graph())
        
        coord = tf.train.Coordinator() 
        threads = tf.train.start_queue_runners( sess = sess, coord = coord )
        

        number_batch = 100 # 应该等于图片总数
        for i in range( number_batch ):
            batch_x, batch_y = sess.run([image_batch, label_batch])
            print(batch_x, batch_y)           
            
            # 开始训练数据，同时训练三个变量，返回三个数据
            _, loss, summary = sess.run([train_step, cross_entropy, merged_summary_op],
                                   feed_dict = {x:batch_x, y_:batch_y, keep_prob_5:0.5, keep_prob_75:0.75})
            summary_writer.add_summary(summary, n * num_batch + i)
            # 打印损失
            # print("loss ", n*num_batch + i, loss)

            if (n * num_batch + i) % 100 == 0:
                # 获取测试数据的准确率
                acc = accuracy.eval({x:test_x, y_:test_y, keep_prob_5:1.0, keep_prob_75:1.0})
                print(n * num_batch + i, "acc:", acc, "  loss:", loss)
                    
                # 准确率大于0.98时保存并退出
                if acc > 0.98 and n > 2:
                    saver.save(sess, model_path + '/train_faces.model', global_step = n * num_batch + i)
                    sys.exit(0)
        
        print('accuracy less 0.98, exited!')
        
        #关闭线程  
        coord.request_stop()  
        coord.join(threads)

        

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.FailedPreconditionError'>, Attempting to use uninitialized value matching_filenames
	 [[Node: matching_filenames/read = Identity[T=DT_STRING, _class=["loc:@matching_filenames"], _device="/job:localhost/replica:0/task:0/cpu:0"](matching_filenames)]]


OutOfRangeError: RandomShuffleQueue '_101_shuffle_batch_8/random_shuffle_queue' is closed and has insufficient elements (requested 5, current size 0)
	 [[Node: shuffle_batch_8 = QueueDequeueManyV2[component_types=[DT_FLOAT, DT_INT64], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](shuffle_batch_8/random_shuffle_queue, shuffle_batch_8/n)]]

Caused by op 'shuffle_batch_8', defined at:
  File "/usr/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/yangyuqi/ve_tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/yangyuqi/ve_tensorflow/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/yangyuqi/ve_tensorflow/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.i